In [1]:
!pwd

/ekaterina/work/src/lca/lca/notebooks


In [12]:
'''
Assign the log file name.
'''

viewpoint = 'right'

clustering_file = f'/ekaterina/work/src/lca/lca/tmp/zebra_drone/ca_filtered_merged_tracking_ids_dji_0100_0101_0102_no_unwanted_viewpoints_pre_miewid_{viewpoint}/clustering.json'
node2uuid_file = f'/ekaterina/work/src/lca/lca/tmp/zebra_drone/ca_filtered_merged_tracking_ids_dji_0100_0101_0102_no_unwanted_viewpoints_pre_miewid_{viewpoint}/node2uuid_file.json'
anno_file = '/ekaterina/work/src/lca/lca/data/drone/ca_filtered_merged_tracking_ids_dji_0100_0101_0102_no_unwanted_viewpoints_pre_miewid.csv.json'

In [13]:
import pandas as pd
import json

def load_json(file_path):
    with open(file_path) as f:
        data = json.load(f)
    return data

data = load_json(anno_file)

dfc = pd.DataFrame(data['categories'])
dfa = pd.DataFrame(data['annotations'])
dfi = pd.DataFrame(data['images'])

df = dfa.merge(dfi, left_on='image_uuid', right_on='uuid')
# df = dfa.merge(dfi, left_on='image_uuid', right_on='uuid')
# df = df.merge(dfc, left_on='id', right_on='id')

# df = df[df['annotations_census'] == True]
df = df[df['viewpoint'] == viewpoint]

counts = df.groupby('viewpoint').size()
print(counts)


viewpoint
right    674
dtype: int64


In [14]:
clusters = load_json(clustering_file)
node2uuid = load_json(node2uuid_file)


uuid_to_cluster = {}


for cluster_id, nodes in clusters.items():
    for node in nodes:
        uuid = node2uuid.get(str(node))  # Get UUID from node2uuid
        if uuid:
            uuid_to_cluster[uuid] = cluster_id  # Map UUID to cluster ID
        

print(uuid_to_cluster)
# print(node2uuid)

{'af34a9d4-edc7-412d-ad60-e700f7b12bb6': 'tc004', '6fec91e4-d7c2-4415-b075-4cb4466dbdfb': 'tc006', '4a211ef0-726d-4ae6-9144-b499e7cfcb26': 'tc008', '75e78b8b-b932-4ca0-b19f-f8f81a1615f8': 'tc019', '693397dc-df1e-4abe-a829-5c256c296101': 'tc019', '36e7fe00-a1c1-4121-b7d2-0e00683383f8': 'tc019', '376a574e-ff09-45a5-8a0d-805c8f3a0b7e': 'tc019', 'c476bffa-2ff9-436e-a7b1-415639439b1b': 'tc019', '78615c0f-55b4-4859-a39f-77094b8830fe': 'tc019', '2388c46b-f7c7-4e54-9dd0-e56adc9a4899': 'tc020', 'b20b0e3a-b8d8-406f-9e85-686f9dabaa77': 'tc021', 'a93386c7-b2d8-42fc-b039-bcf2947d0a33': 'tc022'}


In [15]:
import random
cluster_ids = list(clusters.keys())

# Function to get cluster ID or assign a random cluster if missing
def get_cluster_or_random(uuid):
    return uuid_to_cluster.get(uuid, random.choice(cluster_ids))

# Apply the function to map or assign random values for missing mappings
df['uuid'] = df['uuid_x']
# df['LCA_clustering_id'] = df['uuid'].apply(get_cluster_or_random)

# df['LCA_clustering_id'] = df['uuid'].map(uuid_to_cluster)
df['LCA_clustering_id'] = df['uuid'].map(uuid_to_cluster).where(df['uuid'].isin(uuid_to_cluster), None)


In [17]:
import json
output_path = f'/ekaterina/work/src/lca/lca/data/drone/ca_filtered_merged_tracking_ids_dji_0100_0101_0102_no_unwanted_viewpoints_pre_miewid_{viewpoint}_LCA.json'

# df_annotations_fields = ['uuid', 'image_uuid', 'bbox', 'viewpoint', 'tracking_id', 'individual_id', 'confidence','detection_class','species', 'CA_score', 'category_id', 'LCA_clustering_id']
df_annotations_fields = ['uuid', 'image_uuid', 'bbox', 'viewpoint', 'tracking_id', 'confidence','detection_class','species', 'CA_score', 'category_id', 'LCA_clustering_id']

df_annotations = df[df_annotations_fields]
# df_annotations = df_annotations.rename(columns={'uuid_x': 'uuid'})


df_images_fields = ['image_uuid', 'file_name']
df_images_fields= df.columns.intersection(df_images_fields)
df_images = df[df_images_fields].drop_duplicates(keep='first').reset_index(drop=True)
df_images = df_images.rename(columns={'image_uuid': 'uuid'})


df_categories_fields = ['category_id', 'species']
df_categories = df[df_categories_fields].drop_duplicates(keep='first').reset_index(drop=True)
df_categories = df_categories.rename(columns={'category_id': 'id'})

result_dict = {
                'categories': df_categories.to_dict(orient='records'),
                'images': df_images.to_dict(orient='records'),
                'annotations': df_annotations.to_dict(orient='records')
                }

with open(output_path, 'w') as f:
    json.dump(result_dict, f, indent=4)
    print('Data is saved to:', output_path)


Data is saved to: /ekaterina/work/src/lca/lca/data/drone/ca_filtered_merged_tracking_ids_dji_0100_0101_0102_no_unwanted_viewpoints_pre_miewid_right_LCA.json
